In [1]:
pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
"""
@authors: faurand, chardes, ehagensieker
"""

# all our imports 
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from google.colab import drive
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import datetime
import tqdm
import re


# in a notebook, load the tensorboard extension, not needed for scripts
%load_ext tensorboard

In [3]:
# bash code to mount the drive
drive.mount("/content/drive")
os.chdir("drive/MyDrive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 2.1 the dataset
with open("/content/bible.txt", "r") as f:
  bible = f.read()

# print a short example
print(bible[:500])

The First Book of Moses:  Called Genesis


1:1 In the beginning God created the heaven and the earth.

1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the Spirit of God moved upon the face of the
waters.

1:3 And God said, Let there be light: and there was light.

1:4 And God saw the light, that it was good: and God divided the light
from the darkness.

1:5 And God called the light Day, and the darkness he called Night.
And the evening and the mornin


In [5]:
# exersize 2.2
import tensorflow_text as tf_text
import nltk 

# convert to lower case, remove \n and special characters
bible = bible.replace("\n", " ").lower()
bible = re.sub('[^A-Za-z]+', ' ', bible)
print("converted: ", bible[:500])

# tokenize the text
bible = tf_text.WhitespaceTokenizer().split(bible)
bible = list(bible.numpy().astype('str'))
print("tokenized: ", bible[:500])

converted:  the first book of moses called genesis in the beginning god created the heaven and the earth and the earth was without form and void and darkness was upon the face of the deep and the spirit of god moved upon the face of the waters and god said let there be light and there was light and god saw the light that it was good and god divided the light from the darkness and god called the light day and the darkness he called night and the evening and the morning were the first day and god said let the
tokenized:  ['the', 'first', 'book', 'of', 'moses', 'called', 'genesis', 'in', 'the', 'beginning', 'god', 'created', 'the', 'heaven', 'and', 'the', 'earth', 'and', 'the', 'earth', 'was', 'without', 'form', 'and', 'void', 'and', 'darkness', 'was', 'upon', 'the', 'face', 'of', 'the', 'deep', 'and', 'the', 'spirit', 'of', 'god', 'moved', 'upon', 'the', 'face', 'of', 'the', 'waters', 'and', 'god', 'said', 'let', 'there', 'be', 'light', 'and', 'there', 'was', 'light', 'and', 'god', 'saw'

In [6]:
# counting all words from the corpus and get 10000 most frequent words
words = list(bible)
counts = Counter(words)
print(counts)

most_common = counts.most_common(10000)
words = [x[0] for x in most_common]
print("most common words: ", words)


Counter({'the': 64020, 'and': 51696, 'of': 34668, 'to': 13580, 'that': 12912, 'in': 12667, 'he': 10419, 'shall': 9838, 'unto': 8997, 'for': 8971, 'i': 8854, 'his': 8473, 'a': 8177, 'lord': 7964, 'they': 7376, 'be': 7013, 'is': 6989, 'him': 6659, 'not': 6596, 'them': 6430, 'it': 6129, 'with': 6012, 'all': 5620, 'thou': 5474, 'thy': 4600, 'was': 4522, 'god': 4472, 'which': 4413, 'my': 4368, 'me': 4096, 'said': 3999, 'but': 3992, 'ye': 3983, 'their': 3942, 'have': 3904, 'will': 3836, 'thee': 3827, 'from': 3642, 'as': 3520, 'are': 2950, 'when': 2834, 'this': 2785, 'out': 2775, 'were': 2772, 'upon': 2748, 'man': 2735, 'by': 2625, 'you': 2617, 'israel': 2575, 'king': 2541, 'son': 2392, 'up': 2380, 'there': 2299, 'hath': 2264, 'then': 2169, 'people': 2143, 'came': 2093, 'had': 2026, 'house': 2024, 'into': 2015, 'on': 2011, 'her': 1995, 'come': 1971, 'one': 1969, 'we': 1844, 'children': 1821, 'before': 1796, 's': 1790, 'your': 1785, 'also': 1769, 'day': 1743, 'land': 1718, 'an': 1699, 'so': 16

In [7]:
# apply subsampling to discard words that appear very often
s = 0.001
word_counts = Counter(words)
total_count = 1000

freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - (np.sqrt(freqs[word]/s) + 1) for word in word_counts}
train_words = [word for word in words if random.random() < (1 - p_drop[word])]
print(len(train_words), train_words)

10000 ['the', 'and', 'of', 'to', 'that', 'in', 'he', 'shall', 'unto', 'for', 'i', 'his', 'a', 'lord', 'they', 'be', 'is', 'him', 'not', 'them', 'it', 'with', 'all', 'thou', 'thy', 'was', 'god', 'which', 'my', 'me', 'said', 'but', 'ye', 'their', 'have', 'will', 'thee', 'from', 'as', 'are', 'when', 'this', 'out', 'were', 'upon', 'man', 'by', 'you', 'israel', 'king', 'son', 'up', 'there', 'hath', 'then', 'people', 'came', 'had', 'house', 'into', 'on', 'her', 'come', 'one', 'we', 'children', 'before', 's', 'your', 'also', 'day', 'land', 'an', 'so', 'men', 'against', 'shalt', 'if', 'at', 'let', 'go', 'hand', 'us', 'saying', 'made', 'went', 'no', 'even', 'do', 'now', 'behold', 'saith', 'therefore', 'every', 'these', 'because', 'after', 'our', 'things', 'father', 'down', 'or', 'sons', 'hast', 'o', 'david', 'make', 'say', 'may', 'over', 'did', 'earth', 'what', 'jesus', 'she', 'who', 'great', 'name', 'thine', 'among', 'away', 'any', 'put', 'thereof', 'forth', 'give', 'neither', 'take', 'am', 'd

In [9]:
dict_1 = {i:x for i, x in enumerate(list(train_words))}
dict_2 = {x: i for i, x in enumerate(list(train_words))}

print(dict_1)

# input-target pairs
# tuples = []
# for word in range(1000):
#   for neighbor in range(word-2,word+3):
#     if word!=neighbor and neighbor>=0 and neighbor<1000:
#       tuples.append((dict_1[train_words[word]], dict_1[train_words[neighbor]]))
# print(tuples)

window_size = 4
vocab_size = len(dict_1)

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      bible,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))
for target, context in positive_skip_grams[:6]: 
  print(f"({target}, {context}): ({dict_2[target]}, {dict_2[context]})")

{0: 'the', 1: 'and', 2: 'of', 3: 'to', 4: 'that', 5: 'in', 6: 'he', 7: 'shall', 8: 'unto', 9: 'for', 10: 'i', 11: 'his', 12: 'a', 13: 'lord', 14: 'they', 15: 'be', 16: 'is', 17: 'him', 18: 'not', 19: 'them', 20: 'it', 21: 'with', 22: 'all', 23: 'thou', 24: 'thy', 25: 'was', 26: 'god', 27: 'which', 28: 'my', 29: 'me', 30: 'said', 31: 'but', 32: 'ye', 33: 'their', 34: 'have', 35: 'will', 36: 'thee', 37: 'from', 38: 'as', 39: 'are', 40: 'when', 41: 'this', 42: 'out', 43: 'were', 44: 'upon', 45: 'man', 46: 'by', 47: 'you', 48: 'israel', 49: 'king', 50: 'son', 51: 'up', 52: 'there', 53: 'hath', 54: 'then', 55: 'people', 56: 'came', 57: 'had', 58: 'house', 59: 'into', 60: 'on', 61: 'her', 62: 'come', 63: 'one', 64: 'we', 65: 'children', 66: 'before', 67: 's', 68: 'your', 69: 'also', 70: 'day', 71: 'land', 72: 'an', 73: 'so', 74: 'men', 75: 'against', 76: 'shalt', 77: 'if', 78: 'at', 79: 'let', 80: 'go', 81: 'hand', 82: 'us', 83: 'saying', 84: 'made', 85: 'went', 86: 'no', 87: 'even', 88: 'do

In [15]:
# char_tokens = [dict_1[char] for char in tuples]
# dataset = tf.data.Dataset.from_tensor_slices(char_tokens)
dataset = tf.data.Dataset.from_tensor_slices(positive_skip_grams)

def prepare_data(data, batch_size = 128, sequence_length = 4): 
  # data = data.map(lambda x, t: (tf.cast(x, tf.int64), tf.cast(t, tf.int64)))
  #shuffle, batch, prefetch
  data = data.shuffle(10000)
  data = data.batch(batch_size)
  data = data.prefetch(tf.data.AUTOTUNE)
  
  return data

train_ds = prepare_data(dataset)
for seq in train_ds.take(1):
  tf.print(seq)

[["of" "are"]
 ["not" "the"]
 ["and" "on"]
 ...
 ["secret" "which"]
 ["not" "ground"]
 ["thee" "o"]]


In [21]:
class SkipGram(tf.keras.Model): 

  def __init__(self, num_words, embedding): 

    super(SkipGram, self).__init__()
    self.num_words = num_words
    self.embedding = embedding

  def build(self, input_shape): 
    self.weight_matrix = self.add_weight(shape=(self.num_words, self.embedding), trainable = True)

  @tf.function
  def call(self, inputs, labels): 
    embeddings = tf.nn.embedding_lookup(self.weight_matrix, inputs)
    return embeddings
    

In [17]:
def train_step(model, input, target, optimizer = tf.keras.optimizers.SGD(), num_words = 10000): 

  with tf.GradientTape() as tape: 
    embedding = model(input)
    biases = tf.Variable(tf.zeros([num_words]))
    target = tf.reshape(target, (target.shape[0], 1))
    loss = tf.nn.nce_loss(weights = model.get_weights()[0], biases = biases, labels = target, inputs = embedding, num_classes = num_words)

    loss = tf.reduce_mean(loss)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  return loss

def train_step(self, test_ds): 
  losses = []

  for input, target in test_ds: 
    target = tf.expand_dims(target, -1)
    loss = self(input, target)
    losses.append(loss)
    
  return tf.reduce_mean(losses, axis = 0)
  

In [18]:
def training_loop(model, train_ds, test_ds, optimizer, num_words, epochs = 10): 
  
  for epoch in range(epochs): 

    losses = []

    for input, target in train_ds: 
      train_loss = train_step(model, input, target, optimizer, num_words = num_words)
      losses.append(losses)

    value = model.embedding.numpy()
    cosine = tf.keras.losses.CosineSimilarity()

    for data in test_ds: 
      embedding = model(data)
      cosine = np.ones(value.shape[0])
      


In [22]:
text_words = ["holy, father, wine, poison, love, strong, day"]
num_words = 10000
embedding = 64
num_epochs = 15

model = SkipGram(num_words = num_words, embedding = embedding)
optimizer = tf.keras.optimizers.SGD()

model.compile()
